In [2]:
from time import sleep
from tqdm import tqdm

In [4]:
import numpy as np
import pandas as pd
import requests

In [9]:
tqdm.pandas()


**Мега функция**

Отдает нам район по адресу в формате "улица, город, страна"

In [ ]:
def to_district(address: str, city: str, district_cache) -> str: # фун - я для опр. района по адресу. Используем API OSM
    """
    :address: тут мы поулчаем адрес, уже сформированный правильным образом для передачи в OSM
    :city: передаем город, нужен для cache, чтобы обращаться конкретно к кешу города
    :district_cache: очев. кэш
    """
    url = "https://nominatim.openstreetmap.org/search"
    if city not in district_cache: # для нового города создаем кэш
        district_cache[city] = {}
        
    if address in district_cache[city]: # проверка наличия адреса в кэше города
        return district_cache[city][address]
    
    params = {
      "q": address,
      "format": "json",
      "addressdetails": 1, # input
      "limit": 1 # output
    }
    
    headers = {
        "User-Agent": "RealEstateDistrictFetcher/1.0 (klvalopatin@edu.hse.ru; https://example.com)",
        "Accept-Language": "ru"
    }

    try:
        response = requests.get(url, params=params, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        result = f"Ошибка запроса: {e}"
        district_cache[city][address] = result
        return result

    sleep(0.1)

    if not data:
        result = "Адрес не найден"
        district_cache[city][address] = result
        return result

    address_info = data[0].get('address', {})
    district = (
        address_info.get("city_district") or    
        address_info.get("district") or          
        address_info.get("borough") or           
        address_info.get("municipality") or       
        address_info.get("suburb") or           
        address_info.get("neighbourhood") or     
        address_info.get("quarter") or          
        address_info.get("city_block") or         
        address_info.get("state") or               
        "Район не найден"
    )
    district_cache[city][address] = district
    return district or "Район не найден"

**Сохраняем и создаем общий файл со всеми городами**

In [ ]:
estate_msk = pd.read_csv('../Web-parsing/data_estates/estate_msk.csv')
estate_spb = pd.read_csv('../Web-parsing/data_estates/estate_spb.csv')
estate_novosibirsk = pd.read_csv('../Web-parsing/data_estates/estate_novosibirsk.csv')
estate_nNovg = pd.read_csv('../Web-parsing/data_estates/estate_nizhnij-novgorod.csv')
estate_krasnoyarsk = pd.read_csv('../Web-parsing/data_estates/estate_krasnoyarsk.csv')
estate_chelyabinsk = pd.read_csv('../Web-parsing/data_estates/estate_chelyabinsk.csv')
estate_krasnodar = pd.read_csv('../Web-parsing/data_estates/estate_krasnodar-1.csv')



**Проход по всем городам в отдельности**


Чтобы не работать с digdata

In [ ]:
all_cities = {"ekaterinburg" :"Екатеринбург","chelyabinsk" : "Челябинск", "msk" : "Москва", "spb" : "Санкт-Петербург", "krasnoyarsk" : "Красноярск", "nizhnij-novgorod" : "Нижний Новгород", 
          "novosibirsk" : "Новосибирск"}

In [ ]:
estate_turple = [
    estate_msk,
    estate_spb, 
    estate_novosibirsk,
    estate_nNovg,
    estate_krasnoyarsk,
    estate_chelyabinsk,
    estate_krasnodar
]

In [ ]:
district_cache = {}

In [ ]:
for estate in estate_turple:
    estate['OSM_address'] = estate['full_address'] + ', ' + estate['city'] + ', Россия'
    estate["district"] = estate.progress_apply(
        lambda x: to_district(x['full_address'], x['city'], district_cache),
        axis=1
    )

**Второй споособ: формируем в единый датасет, проходимся по нему и определяем район**

In [ ]:
estate_cities = pd.concat([
    estate_msk,
    estate_spb,
    estate_novosibirsk,
    estate_krasnodar,
    estate_chelyabinsk,
    estate_nNovg,
    estate_krasnoyarsk
])
estate_cities = estate_cities.reset_index(drop=True)

In [60]:
estate_cities['OSM_address'] = estate_cities['full_address'] + ', ' + estate_cities['city'] + ', Россия'

In [ ]:
district_cache = {}


In [ ]:
estate_cities["district"] = estate_cities.progress_apply( # мега функция для всех 13к строк (район создаем в табличку)
    lambda x: to_district(x['full_address'], x['city'], district_cache),
    axis=1
)

100%|██████████| 13866/13866 [2:37:37<00:00,  1.47it/s]  


In [42]:
district_cache_temp = {}

In [40]:
estate_ekaterinburg= pd.read_csv('../Web-parsing/data_estates/estate_ekaterinburg.csv')

In [41]:
estate_ekaterinburg['OSM_address'] = estate_ekaterinburg['full_address'] + ', ' + estate_ekaterinburg['city'] + ', Россия'

In [43]:
estate_ekaterinburg["district"] = estate_ekaterinburg.progress_apply( # мега функция для всех 13к строк (район создаем в табличку)
    lambda x: to_district(x['full_address'], x['city'], district_cache_temp),
    axis=1
)

100%|██████████| 1205/1205 [05:15<00:00,  3.82it/s]


In [48]:
estate_cities = pd.concat([
    estate_cities,
    estate_ekaterinburg
])

In [ ]:
def to_normAdress(address: str):
    return address.split(',')[0]



In [14]:
mask = estate_cities["district"] == "Адрес не найден"

estate_cities.loc[mask, "district"] = estate_cities.loc[mask].progress_apply(
    lambda x: to_district(to_normAdress(x["full_address"]), x["city"], district_cache),
    axis=1
)


100%|██████████| 2424/2424 [10:21<00:00,  3.90it/s] 


In [37]:
estate_cities = estate_cities.drop("Unnamed: 0", axis=1).reset_index(drop=True)

In [ ]:
estate_cities = estate_cities[estate_cities['city'] != 'Краснодар']

In [ ]:
estate_cities.to_csv("../Web-parsing/data_estates/estate_cities.csv", index=False) # сохр


In [6]:
estate_cities

,"rent, rub/month",full_address,city,"price_per_sqm, rub/month","square, m^2",type,url,OSM_address,district
0,450000,"улица Шумкина, 20 ст1",Москва,1698,265,Универсальное помещение,https://msk.n1.ru/view/115904136/?open_card_kn,"улица Шумкина, 20 ст1, Москва, Россия",район Сокольники
1,4097925,"улица Берсеневская набережная, 8 ст1",Москва,3249,1260,Универсальное помещение,https://msk.n1.ru/view/115813194/?open_card_kn,"улица Берсеневская набережная, 8 ст1, Москва, ...",Адрес не найден
2,68125,"улица Шумкина, 20 ст1",Москва,2084,32,Универсальное помещение,https://msk.n1.ru/view/115777113/?open_card_kn,"улица Шумкина, 20 ст1, Москва, Россия",район Сокольники
3,65000,"улица Николая Озерова, 2",Москва,4333,15,Универсальное помещение,https://msk.n1.ru/view/115738288/?open_card_kn,"улица Николая Озерова, 2, Москва, Россия",район Покровское-Стрешнево
4,490000,"улица Новослободская, 50 ст2",Москва,1580,310,Универсальное помещение,https://msk.n1.ru/view/115137027/?open_card_kn,"улица Новослободская, 50 ст2, Москва, Россия",Тверской район
...,...,...,...,...,...,...,...,...,...
14575,100000,Громова,Екатеринбург,1000,100,Помещение под бар/ресторан,https://ekaterinburg.n1.ru/view/115624215/?ope...,"Громова, Екатеринбург, Россия",Новогромовское сельское поселение
14576,150000,Краснофлотцев,Екатеринбург,1428,105,Помещение под бар/ресторан,https://ekaterinburg.n1.ru/view/114934584/?ope...,"Краснофлотцев, Екатеринбург, Россия",Орджоникидзевский район
14577,199500,Восточная,Екатеринбург,750,266,Помещение под бар/ресторан,https://ekaterinburg.n1.ru/view/113846166/?ope...,"Восточная, Екатеринбург, Россия",Репьёвский сельсовет
14578,85000,Фурманова,Екатеринбург,1562,54,Помещение под бар/ресторан,https://ekaterinburg.n1.ru/view/115608931/?ope...,"Фурманова, Екатеринбург, Россия",Алматинская область
